In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [3]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils
from sklearn.metrics import confusion_matrix, f1_score, auc, roc_curve, precision_recall_curve

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

# Language Model

In [5]:
lm = True
clean = False

In [6]:
if lm: path = Path('../input'); 
list(path.iterdir())

[WindowsPath('../input/embeddings'),
 WindowsPath('../input/glove.840B.300d'),
 WindowsPath('../input/sample_submission.csv'),
 WindowsPath('../input/test.csv'),
 WindowsPath('../input/test_Clean.csv'),
 WindowsPath('../input/tmp_lm'),
 WindowsPath('../input/train.csv'),
 WindowsPath('../input/train_Clean.csv')]

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
allText_df = train_df.copy().append(test_df.copy(), sort=False)

In [7]:
if clean:
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    
    train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
    test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: x.lower())

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(x))
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_text(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_numbers(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x:clean_numbers(x))
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_numbers(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: replace_typical_misspell(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: replace_typical_misspell(x))
    #allText_df["question_text"] = allText_df["question_text"].apply(lambda x: replace_typical_misspell(x))

    train_x = train_df['question_text'].fillna('_##_').values
    test_x = test_df['question_text'].fillna('_##_').values
    
    train_Full = pd.read_csv(path/'train.csv')
    test_Full = pd.read_csv(path/'test.csv')
    
    train_Full['question_text'] = train_x
    test_Full['question_text'] = test_x
    
    train_Full.to_csv('train_Clean.csv',index=False)
    test_Full.to_csv('test_Clean.csv',index=False)

import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [8]:
#print(df_train.head(3))

print(train_df.shape)
print(test_df.shape)

In [9]:
# df_train.head(3)

In [10]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = path/'train_Clean.csv',
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT)] 
    test = torchtext.data.TabularDataset( 
                path=path/'test_Clean.csv',
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [11]:
TEXT,LABEL,train,test = prepare_sequences()


LABEL.batch_first


In [12]:
TEXT.build_vocab(train, test, vectors = "glove.6B.50d")

In [13]:
train_storage = train

In [14]:
#train = train_storage

In [15]:
train,vali = train.split(0.8)

In [36]:
from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size= 512, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)

vali_iter = BucketIterator(
    vali, # we pass in the datasets we want the iterator to draw data from
    batch_size= 512, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=True)

test_iter = Iterator(
    test,
    batch_size = 512,
    train=False,
    sort = False,
    sort_within_batch = False,
    repeat = False)

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

iter(test_iter).__next__().question_text

#len(train_iter)
#label_Size = next(iter(train_iter)).target
#len(label_Size[label_Size==1])
#train_iter.batches
#train_iter = iter(train_iter)
#train_iter.__next__().question_text
#train_iter.__next__().target
#iter(train_iter).__next__().question_text.shape

#smaller,_ = train.split(0.1)
TEXT.vocab.itos[10]

test_iter.dataset.examples.__getitem__(1).__dict__

#smaller.examples[0].question_text
test.examples.__getitem__(2).__dict__

In [17]:
#len(train)
#len(test_iter)

In [18]:
#train.examples[0].question_text

In [19]:
TEXT.vocab.vectors.shape

torch.Size([214945, 50])

TEXT.vocab.freqs

In [20]:
embed_size = 50
hidden_size = 300
ver1 = False
sigBCE = True
bidir = True
layer_LSTM = 2
layer_Lin = 3
dropout_LSTM = 0.1
dropout_Lin = 0.1

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        
        hidden_size = 64
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size*4,2)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(16,1)
        
    def forward(self,train):
        #print(train.shape)
        h_embedding = self.embedding(train)
        #print(h_embedding.shape)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding,0)))
        #print(h_embedding.shape)
        
        h_lstm, _ = self.lstm(h_embedding)
        #print(h_lstm.shape)
        h_gru, _ = self.gru(h_lstm)
        #print(h_gru.shape)
        
        avg_pool = torch.mean(h_gru,1)
        #print(avg_pool.shape)
        #avg_pool = avg_pool.view(-1,64)
        max_pool, _ = torch.max(h_gru,1)
        #print(max_pool.shape)
        
        
        conc = torch.cat((avg_pool, max_pool),1)
        print(conc.shape)
        conc = self.relu(self.linear(conc))
        #print(conc.shape)
        #conc = self.dropout(conc)
        print(conc.shape)
        #out = nn.Sigmoid()(conc)
        
        out = nn.LogSoftmax(dim=1)(conc)
        
        #out = self.out(conc)
        #print(out[:,-1].shape)
        
        return out
    
    
    
    
batch = next(iter(train_iter)).question_text
net = NeuralNet()
#net(batch)

In [21]:
#batch = next(iter(train_iter)).question_text
#batch[30]
#layer_Lin

In [22]:
class LSTMTagger(nn.Module):
    def __init__(self):
        super(LSTMTagger, self).__init__()
                      
        self.hidden_dim = hidden_size
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        
        
        #self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm_Layer = nn.LSTM(embed_size, hidden_size,num_layers = layer_LSTM, dropout = dropout_LSTM, bidirectional = bidir)
        
        self.lin_Layer = []
        
        self.lin_Layer_Dropout = dropout_Lin
        
        self.relu = nn.ReLU()
        
        if bidir:
            scale = 2
        else:
            scale = 1
        
        for _ in range(layer_Lin - 1):
            self.lin_Layer.append(nn.Linear(hidden_size*scale,hidden_size*scale))
            self.lin_Layer = nn.ModuleList(self.lin_Layer)
        if sigBCE:
            self.hidden2tag = nn.Linear(hidden_size*scale, 1)
        else:
            self.hidden2tag = nn.Linear(hidden_size*scale, 2)
    
    def forward(self, train):
        #print(train.shape)
        embeds = self.embedding(train)
        #print(embeds.shape)
        #print(embeds.view(len(train),64,-1).shape)
        #print(embeds.view(len(train),train.shape[1],-1).shape)
        lstm_out, _ = self.lstm_Layer(embeds)#embeds.view(len(train),train.shape[1],-1))
        #print(lstm_out.shape)
        
        if ver1:
            #print(lstm_out.shape)
            feature = lstm_out[-1,:,:] # -1 grabs last layer in list
            #print(feature.shape)
        
            for layer in self.lin_Layer:
                feature = layer(feature)
                feature = F.relu(feature)
                predict = self.hidden2tag(feature)
        
            feature = F.relu(feature)
            predict = self.hidden2tag(feature)
            if sigBCE:
                return nn.Sigmoid()(predict)
            else:
                return nn.LogSoftmax(dim=1)(predict)
        else:
           
            feature = torch.mean(lstm_out,0) #avg_pool
            #print(avg_pool.shape)
            for layer in self.lin_Layer:
                feature = layer(feature)
                feature = F.relu(feature)
                
            target_space = self.hidden2tag(feature)#.view(len(train),-1))
            #target_space = self.relu(target_space)
            
            #print(target_space.shape)
            if sigBCE:
                return nn.Sigmoid()(target_space)
            else:
                return F.log_softmax(target_space, dim=1)
            #print(target_score.shape)

inputs_check = next(iter(train_iter)).question_text

#print(inputs_check.shape)

model = LSTMTagger()

with torch.no_grad():
    inputs = inputs_check
    #print(len(inputs))
    score = model(inputs)
    print(score.shape)
    print(score)

torch.Size([128, 1])
tensor([[0.4897],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4895],
        [0.4897],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4894],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4896],
        [0.4895],
        [0.4897],
        [0.4897],
        [0.4895],
        [0.4896],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4897],
        [0.4896],
        [0.4895],
        [0.4894],
        [0.4895],
        [0.4898],
        [0.4895],
        [0.4897],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4895],
        [0.4897],
        [0.4895],
        [0.4895],
        [0.4896],
        [0.4895],
        [0.4896],
        [0.4894],
        [0.4894],
       

In [23]:
def weight_init(m):
    if  isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

In [24]:
model = LSTMTagger()
model.apply(weight_init)
if sigBCE:
    loss_function = nn.BCELoss(reduction='none')
else:
    loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.2, momentum=0.9)

In [ ]:
validate = True

checking = next(iter(train_iter))
inputs_check = checking.question_text
target_check = checking.target


f1_train = []
auc_train = []
train_accu = []

avg_loss = 0

with torch.no_grad():
    inputs = inputs_check
    score1 = model(inputs)
    #print(score)
    
for epoch in range(2):
    model.train()
    avg_loss = 0
    training_set = iter(train_iter)
    vali_set = iter(vali_iter)
    model.zero_grad()
    all_preds = []
    all_actual = []
    for batch_num in range(len(train_iter)):
        model.train()
        sentence_data = next(training_set)
        vali_data = next(vali_set)
        sentence_in = sentence_data.question_text
        vali_in = vali_data.question_text
        
        
        #target -> target.unsqueeze(1)
        if sigBCE:
            targets = sentence_data.target.float()#.unsqueeze(1)
            vali_target = vali_data.target.float()
        else:
            targets = torch.LongTensor(sentence_data.target)#.unsqueeze(1)
            vali_target = torch.LongTensor(vali.target)
        optimizer.zero_grad()
        
        target_scores = model(sentence_in)
        
        loss = loss_function(target_scores.squeeze(1), targets)#.squeeze(1))#.reshape(64,1).squeeze(1))
        loss = (loss * (targets*17+1)/(targets*17+1).sum()).sum()#.mean()
        loss.backward()
        optimizer.step()
        
        
        model.eval()
        vali_scores = model(vali_in)
        vali_loss = loss_function(vali_scores.squeeze(1), vali_target).mean()
        vali_f1 = (np.asarray(vali_scores.squeeze(1)) > 0.5).astype(int)
        if sigBCE:
            #prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
            #accuracy = np.asarray(prediction == np.asarray(targets)).sum()/len(targets)*100
            prediction = np.asarray(vali_scores.squeeze(1)>0.5,dtype=float)
            accuracy = np.asarray(prediction == np.asarray(vali_target)).sum()/len(vali_target)*100
        else:
            #prediction = target_scores.data.max(1)[1]
            #accuracy = np.asarray(prediction==targets).sum()/len(targets)*100
            prediction = vali_scores.data.max(1)[1]
            accuracy = np.asarray(prediction == vali_target).sum()/len(vali_target)*100
        #train_accu.append(accuracy)
        
        if sigBCE:
            all_preds.extend(target_scores.tolist())#torch.round(target_scores).tolist())
        else:
            all_preds.extend(torch.exp(target_scores).tolist())#torch.round(target_scores).tolist())
        all_actual.extend(targets.tolist())
        
        #print(np.asarray(target_scores)[:,1].max())
        
        #targets -> targets.reshape(64,1).squeeze(1)
        
        avg_loss += loss.item()
        
        #print(loss)
        if sigBCE:
            print(batch_num+1," of: ",len(train_iter), " Loss: ",loss.item()," F1 Val: ",f1_score(vali_f1, vali_target)," Val Acc: ",accuracy, " High: ", np.asarray(target_scores).max()," Low: ", np.asarray(target_scores).min())
        else:
            print(batch_num+1," of: ",len(train_iter), " Loss: ",loss.item()," Val Loss: ",vali_loss.item()," Acc: ",accuracy, " High: ", np.asarray(torch.exp(target_scores))[:,1].max()," Low: ", np.asarray(target_scores).min())
        
        
    
    #model.eval()
    avg_loss = avg_loss/len(train_iter)
    
    #if validate:
        #avg_val
    
    
    #else:
    print("Training loss for epoch {} is {}".format(epoch + 1, avg_loss))
    
    
    #train_confusion_matrix = confusion_matrix(all_actual, all_preds).ravel()
    if sigBCE:
        pred_Clas = (np.asarray(all_preds) > 0.5).astype(int)#-2.65).astype(int)#
    else:
        pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)#-2.65).astype(int)#
    train_confusion_matrix = confusion_matrix(all_actual, pred_Clas).ravel()
    
    print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
    f1_train.append(f1_score(all_actual, pred_Clas))
    print("F1 score for epoch {} is {}".format(epoch + 1, f1_train[-1]))
    fpr, tpr, _ = roc_curve(all_actual, pred_Clas, pos_label=1)
    auc_train.append(auc(fpr, tpr))
    print("AUC for epoch {} is {}\n".format(epoch + 1, auc_train[-1]))

with torch.no_grad():
    inputs = inputs_check
    score2 = model(inputs)
    #print(score)
        
        

1  of:  2041  Loss:  0.3863082528114319  F1 Val:  0.3414634146341463  Val Acc:  73.6328125  High:  0.9941413  Low:  0.108024865
2  of:  2041  Loss:  0.36439114809036255  F1 Val:  0.3711340206185567  Val Acc:  76.171875  High:  0.9981318  Low:  0.11055037
3  of:  2041  Loss:  0.30511441826820374  F1 Val:  0.31521739130434784  Val Acc:  75.390625  High:  0.9967886  Low:  0.111272566
4  of:  2041  Loss:  0.38811060786247253  F1 Val:  0.5131578947368421  Val Acc:  85.546875  High:  0.99779403  Low:  0.109010965
5  of:  2041  Loss:  0.33682185411453247  F1 Val:  0.43661971830985913  Val Acc:  84.375  High:  0.9960418  Low:  0.106019445
6  of:  2041  Loss:  0.30062249302864075  F1 Val:  0.48437499999999994  Val Acc:  87.109375  High:  0.994361  Low:  0.10217618
7  of:  2041  Loss:  0.27009639143943787  F1 Val:  0.5038167938931297  Val Acc:  87.3046875  High:  0.9722261  Low:  0.09857278
8  of:  2041  Loss:  0.305858314037323  F1 Val:  0.5441176470588235  Val Acc:  87.890625  High:  0.9767293

In [ ]:
vali_data

In [ ]:
torch.save(model,"C:/Users/xw5735/PWinFolder/WPy-3661/notebooks/KaggleChal/LSTMTaggerFull22.pt")

model = torch.load("C:/Users/xw5735/PWinFolder/WPy-3661/notebooks/KaggleChal/LSTMTaggerFull.pt")

In [ ]:
batch_size = 10
nb_classes = 2
target = torch.LongTensor(batch_size).random_(nb_classes)
#print((targets*5+1))
print((loss * (targets*5+1)).mean())

In [ ]:
prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
target_scores

In [ ]:
#preddd = np.asarray(all_preds)[:,1]
#preddd_clas = (preddd>-0.45).astype(int)
#train_confusion_matrix = confusion_matrix(all_actual, preddd_clas).ravel()
#print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
#pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
#f1_score(all_actual, pred_Clas)
#pred_Clas

In [ ]:
#preddd = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
#len(preddd[preddd>0])

In [ ]:
def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result

In [ ]:
def threshold_search_2(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [ ]:
#threshold_search_2(all_actual, preddd)
#np.asarray(all_preds)[:,1].max()

In [ ]:
#preddd = np.asarray(all_preds)[:,1]
#preddd_clas = (preddd>-0.45).astype(int)
#preddd.max()
#len(preddd_clas[preddd_clas == 1])

In [ ]:
#thr = threshold_search_2(all_actual, np.asarray(all_preds));thr

In [ ]:
#thr = threshold_search(all_actual, np.asarray(all_preds), plot=True);thr

In [26]:
all_final_preds = []
all_final_rounding = []
model.eval()
testing_batches = iter(test_iter)
for batch_num in range(len(test_iter)):
    final_preds = model(next(testing_batches).question_text)
    prediction = np.asarray(final_preds>0.5,dtype=float)
    print(batch_num+1,"out of: ",len(test_iter)," max: ",np.asarray(final_preds).max())
    all_final_preds.extend(final_preds.tolist())
    all_final_rounding.extend(prediction.tolist())
    

#probab = np.asarray(final_preds)[:,1]

#final_preds_clas = (probab>thr['threshold']).astype(int)

#final_preds_clas.shape

1 out of:  2936  max:  0.99530315
2 out of:  2936  max:  0.9774315
3 out of:  2936  max:  0.99973136
4 out of:  2936  max:  0.9411771
5 out of:  2936  max:  0.9876131
6 out of:  2936  max:  0.90058154
7 out of:  2936  max:  0.9812444
8 out of:  2936  max:  0.99021214
9 out of:  2936  max:  0.9792358
10 out of:  2936  max:  0.99660194
11 out of:  2936  max:  0.9907809
12 out of:  2936  max:  0.9978168
13 out of:  2936  max:  0.9884581
14 out of:  2936  max:  0.974744
15 out of:  2936  max:  0.99127895
16 out of:  2936  max:  0.99543875
17 out of:  2936  max:  0.9988231
18 out of:  2936  max:  0.9741199
19 out of:  2936  max:  0.9827916
20 out of:  2936  max:  0.9864871
21 out of:  2936  max:  0.98682785
22 out of:  2936  max:  0.9923308
23 out of:  2936  max:  0.98535746
24 out of:  2936  max:  0.9997427
25 out of:  2936  max:  0.9999598
26 out of:  2936  max:  0.9958425
27 out of:  2936  max:  0.98008406
28 out of:  2936  max:  0.990897
29 out of:  2936  max:  0.95975137
30 out of:  29

235 out of:  2936  max:  0.99685884
236 out of:  2936  max:  0.990927
237 out of:  2936  max:  0.99495536
238 out of:  2936  max:  0.99913293
239 out of:  2936  max:  0.99825555
240 out of:  2936  max:  0.9823412
241 out of:  2936  max:  0.9748917
242 out of:  2936  max:  0.9989384
243 out of:  2936  max:  0.95390457
244 out of:  2936  max:  0.98350304
245 out of:  2936  max:  0.99781835
246 out of:  2936  max:  0.9807207
247 out of:  2936  max:  0.97854245
248 out of:  2936  max:  0.99827313
249 out of:  2936  max:  0.9809427
250 out of:  2936  max:  0.99547946
251 out of:  2936  max:  0.96852666
252 out of:  2936  max:  0.99720335
253 out of:  2936  max:  0.9958371
254 out of:  2936  max:  0.9783553
255 out of:  2936  max:  0.9997423
256 out of:  2936  max:  0.96278745
257 out of:  2936  max:  0.96454996
258 out of:  2936  max:  0.97528195
259 out of:  2936  max:  0.989377
260 out of:  2936  max:  0.9754186
261 out of:  2936  max:  0.98453
262 out of:  2936  max:  0.99670994
263 out 

466 out of:  2936  max:  0.9901722
467 out of:  2936  max:  0.99730647
468 out of:  2936  max:  0.94493353
469 out of:  2936  max:  0.99746776
470 out of:  2936  max:  0.99599564
471 out of:  2936  max:  0.9668846
472 out of:  2936  max:  0.98900104
473 out of:  2936  max:  0.9923481
474 out of:  2936  max:  0.9672171
475 out of:  2936  max:  0.98745793
476 out of:  2936  max:  0.95737493
477 out of:  2936  max:  0.9884089
478 out of:  2936  max:  0.9557904
479 out of:  2936  max:  0.9970258
480 out of:  2936  max:  0.9963685
481 out of:  2936  max:  0.99879766
482 out of:  2936  max:  0.99018735
483 out of:  2936  max:  0.9818671
484 out of:  2936  max:  0.9943175
485 out of:  2936  max:  0.99395376
486 out of:  2936  max:  0.99003696
487 out of:  2936  max:  0.9419912
488 out of:  2936  max:  0.95330983
489 out of:  2936  max:  0.99821305
490 out of:  2936  max:  0.98667634
491 out of:  2936  max:  0.9861248
492 out of:  2936  max:  0.99567044
493 out of:  2936  max:  0.99680954
494 

697 out of:  2936  max:  0.99885046
698 out of:  2936  max:  0.9891628
699 out of:  2936  max:  0.97780275
700 out of:  2936  max:  0.9914875
701 out of:  2936  max:  0.9884288
702 out of:  2936  max:  0.9982145
703 out of:  2936  max:  0.9624207
704 out of:  2936  max:  0.98767555
705 out of:  2936  max:  0.9532364
706 out of:  2936  max:  0.9894461
707 out of:  2936  max:  0.99808216
708 out of:  2936  max:  0.99544746
709 out of:  2936  max:  0.9459586
710 out of:  2936  max:  0.97285575
711 out of:  2936  max:  0.9705442
712 out of:  2936  max:  0.99859303
713 out of:  2936  max:  0.9929595
714 out of:  2936  max:  0.986206
715 out of:  2936  max:  0.995993
716 out of:  2936  max:  0.99741024
717 out of:  2936  max:  0.9700931
718 out of:  2936  max:  0.99605787
719 out of:  2936  max:  0.9959805
720 out of:  2936  max:  0.9985442
721 out of:  2936  max:  0.97515917
722 out of:  2936  max:  0.98107374
723 out of:  2936  max:  0.9951025
724 out of:  2936  max:  0.98348236
725 out of

929 out of:  2936  max:  0.997609
930 out of:  2936  max:  0.98717916
931 out of:  2936  max:  0.98582053
932 out of:  2936  max:  0.986798
933 out of:  2936  max:  0.99935335
934 out of:  2936  max:  0.99815995
935 out of:  2936  max:  0.99958795
936 out of:  2936  max:  0.992262
937 out of:  2936  max:  0.9975132
938 out of:  2936  max:  0.98522633
939 out of:  2936  max:  0.99654776
940 out of:  2936  max:  0.99497104
941 out of:  2936  max:  0.9896735
942 out of:  2936  max:  0.9666972
943 out of:  2936  max:  0.9944878
944 out of:  2936  max:  0.98222995
945 out of:  2936  max:  0.9764738
946 out of:  2936  max:  0.9560454
947 out of:  2936  max:  0.97412217
948 out of:  2936  max:  0.98320466
949 out of:  2936  max:  0.98997444
950 out of:  2936  max:  0.9756261
951 out of:  2936  max:  0.9973015
952 out of:  2936  max:  0.9976228
953 out of:  2936  max:  0.9772474
954 out of:  2936  max:  0.9892909
955 out of:  2936  max:  0.99730587
956 out of:  2936  max:  0.99943787
957 out o

...

1831 out of:  2936  max:  0.9704553
1832 out of:  2936  max:  0.95732015
1833 out of:  2936  max:  0.9878298
1834 out of:  2936  max:  0.9723327
1835 out of:  2936  max:  0.99047333
1836 out of:  2936  max:  0.9810362
1837 out of:  2936  max:  0.98401546
1838 out of:  2936  max:  0.9838561
1839 out of:  2936  max:  0.98575056
1840 out of:  2936  max:  0.9958099
1841 out of:  2936  max:  0.9530474
1842 out of:  2936  max:  0.97926635
1843 out of:  2936  max:  0.9893805
1844 out of:  2936  max:  0.9959139
1845 out of:  2936  max:  0.98069566
1846 out of:  2936  max:  0.99859375
1847 out of:  2936  max:  0.99290705
1848 out of:  2936  max:  0.99691474
1849 out of:  2936  max:  0.9968381
1850 out of:  2936  max:  0.9926099
1851 out of:  2936  max:  0.9876616
1852 out of:  2936  max:  0.99500555
1853 out of:  2936  max:  0.999233
1854 out of:  2936  max:  0.9764222
1855 out of:  2936  max:  0.9531229
1856 out of:  2936  max:  0.9937359
1857 out of:  2936  max:  0.99641186
1858 out of:  2936

2057 out of:  2936  max:  0.99938107
2058 out of:  2936  max:  0.9995741
2059 out of:  2936  max:  0.91739374
2060 out of:  2936  max:  0.97992593
2061 out of:  2936  max:  0.95711875
2062 out of:  2936  max:  0.9960914
2063 out of:  2936  max:  0.9631437
2064 out of:  2936  max:  0.98546535
2065 out of:  2936  max:  0.9927295
2066 out of:  2936  max:  0.974681
2067 out of:  2936  max:  0.95248437
2068 out of:  2936  max:  0.99935085
2069 out of:  2936  max:  0.99603397
2070 out of:  2936  max:  0.9956785
2071 out of:  2936  max:  0.99129725
2072 out of:  2936  max:  0.9681692
2073 out of:  2936  max:  0.99246734
2074 out of:  2936  max:  0.9895314
2075 out of:  2936  max:  0.99429697
2076 out of:  2936  max:  0.9889791
2077 out of:  2936  max:  0.98559815
2078 out of:  2936  max:  0.9907211
2079 out of:  2936  max:  0.9825806
2080 out of:  2936  max:  0.94180465
2081 out of:  2936  max:  0.99073863
2082 out of:  2936  max:  0.99081814
2083 out of:  2936  max:  0.96229184
2084 out of: 

2282 out of:  2936  max:  0.9923988
2283 out of:  2936  max:  0.99949896
2284 out of:  2936  max:  0.9899884
2285 out of:  2936  max:  0.937241
2286 out of:  2936  max:  0.99535394
2287 out of:  2936  max:  0.9861135
2288 out of:  2936  max:  0.9757947
2289 out of:  2936  max:  0.9760109
2290 out of:  2936  max:  0.98724306
2291 out of:  2936  max:  0.94277316
2292 out of:  2936  max:  0.9926717
2293 out of:  2936  max:  0.9782356
2294 out of:  2936  max:  0.9719615
2295 out of:  2936  max:  0.9575091
2296 out of:  2936  max:  0.99737763
2297 out of:  2936  max:  0.99106115
2298 out of:  2936  max:  0.99593335
2299 out of:  2936  max:  0.99829644
2300 out of:  2936  max:  0.982781
2301 out of:  2936  max:  0.9965167
2302 out of:  2936  max:  0.9827666
2303 out of:  2936  max:  0.9663842
2304 out of:  2936  max:  0.9975988
2305 out of:  2936  max:  0.9951598
2306 out of:  2936  max:  0.9912816
2307 out of:  2936  max:  0.9746663
2308 out of:  2936  max:  0.9726318
2309 out of:  2936  ma

2507 out of:  2936  max:  0.9974738
2508 out of:  2936  max:  0.98521835
2509 out of:  2936  max:  0.9994122
2510 out of:  2936  max:  0.99690175
2511 out of:  2936  max:  0.9772096
2512 out of:  2936  max:  0.9850032
2513 out of:  2936  max:  0.9619041
2514 out of:  2936  max:  0.9869898
2515 out of:  2936  max:  0.9188766
2516 out of:  2936  max:  0.95640206
2517 out of:  2936  max:  0.9845114
2518 out of:  2936  max:  0.97447217
2519 out of:  2936  max:  0.9543112
2520 out of:  2936  max:  0.9763474
2521 out of:  2936  max:  0.9570099
2522 out of:  2936  max:  0.91049796
2523 out of:  2936  max:  0.9697213
2524 out of:  2936  max:  0.9797702
2525 out of:  2936  max:  0.9617979
2526 out of:  2936  max:  0.98758894
2527 out of:  2936  max:  0.9562008
2528 out of:  2936  max:  0.98027545
2529 out of:  2936  max:  0.98439056
2530 out of:  2936  max:  0.99980575
2531 out of:  2936  max:  0.965693
2532 out of:  2936  max:  0.98602074
2533 out of:  2936  max:  0.99149996
2534 out of:  2936

2732 out of:  2936  max:  0.98870695
2733 out of:  2936  max:  0.9828977
2734 out of:  2936  max:  0.9818503
2735 out of:  2936  max:  0.9963381
2736 out of:  2936  max:  0.9912847
2737 out of:  2936  max:  0.9553376
2738 out of:  2936  max:  0.986013
2739 out of:  2936  max:  0.9945562
2740 out of:  2936  max:  0.9894944
2741 out of:  2936  max:  0.9945952
2742 out of:  2936  max:  0.9959543
2743 out of:  2936  max:  0.9725992
2744 out of:  2936  max:  0.9658352
2745 out of:  2936  max:  0.9854575
2746 out of:  2936  max:  0.99862766
2747 out of:  2936  max:  0.9990183
2748 out of:  2936  max:  0.98593116
2749 out of:  2936  max:  0.9772652
2750 out of:  2936  max:  0.9769255
2751 out of:  2936  max:  0.9802942
2752 out of:  2936  max:  0.9835908
2753 out of:  2936  max:  0.98470324
2754 out of:  2936  max:  0.99583083
2755 out of:  2936  max:  0.984041
2756 out of:  2936  max:  0.997543
2757 out of:  2936  max:  0.98737156
2758 out of:  2936  max:  0.9833363
2759 out of:  2936  max: 

In [27]:
sub = pd.read_csv(path/'sample_submission.csv')
sub['prediction'] = (np.asarray(all_final_preds)>0.5).astype(int)
sub.to_csv("submission_Kag_Full4.csv",index=False)

# Submission Encoding/Decoding

In [ ]:
import os
#os.chdir(os.getcwd()+'/KaggleChal')
path2 = os.getcwd()

sub = pd.read_csv(path2+'/submission_Kag_Full3.csv')
pred_str = ''.join(str(x) for x in list(sub.prediction.values))
print(len(pred_str))

sbase = "".join(chr(i) for i in range(0,65535) if chr(i).isalnum())

def encode(bin_str, sbase):
    n = int(bin_str,2)   
    code = []
    base = len(sbase)
    while n > 0:
        n, c = divmod(n, base)
        code.append(sbase[c])
    code = reversed(code)
    return ''.join(code)

def decode(code, sbase, size):
    code = reversed(code)
    base = len(sbase)
    bin_str =  bin(sum([sbase.index(c)*base**i for i, c in enumerate(code)]))[2:]
    bin_str = (size-len(bin_str))*'0' + bin_str
    return bin_str
# Compression using conversion to hexadecimal base
nhex = hex(int(pred_str,2))
assert int(pred_str,2) == int(nhex,16)
len(nhex)

# Compression using a big base
pred_compress = encode(pred_str, sbase)
len(pred_compress)

print(pred_compress)

In [ ]:
np.asarray(all_final_preds).max()

In [ ]:
testing_batches = iter(test_iter)

In [ ]:
for i, weop in enumerate(test_iter):
    print(i," ",weop.question_text)

In [ ]:
enumerate(test_iter)

In [ ]:
itering = next(testing_batches)

In [ ]:
itering.input_fields

In [ ]:
itering.question_text.shape

In [ ]:
TEXT.vocab.itos[3]

In [ ]:
#thr['threshold']

def train_model(model, train_data, epochs, show=False):
    
    #optimizer = torch.optim.Adam(model.parameters())
    optimizer = optim.SGD(model.parameters(), lr=0.0001)
    #loss_function = nn.BCEWithLogitsLoss(reduction='mean')
    loss_function = nn.NLLLoss()
    
    print("The loss function being used is {}".format(loss_function))
    errors = []
    eval_errors = []
    f1_train = []
    auc_train = []
    f1_eval = []
    auc_eval = []
    
    num_training_batches = len(train_data)
    
    for epoch in range(epochs):
        print("Epoch {}".format(epoch + 1))
        print("Training mode")
        model.train()
        train_iter = iter(train_data)
        model.zero_grad()
        train_loss = 0
        all_preds = []
        all_actual = []
        for batch_num in range(num_training_batches):
            print("Batch {}".format(batch_num + 1))
            print("Out of {}".format(num_training_batches))
            batch = next(train_iter)
            text, class_vector = batch.question_text.transpose(0,1),batch.target.unsqueeze(1)
            #print(batch)
            #print(text.shape)
            #optimizer.zero_grad()
            text_pred = model(text)
            #print(text_pred)
            print(class_vector.shape)
            print(torch.max(class_vector,1)[0].shape)
            loss = loss_function(text_pred,class_vector.reshape(64,1).squeeze(1)) #torch.max(class_vector,1)[0])
            print(loss)
            print(text_pred.squeeze(1))
            print(torch.max(class_vector,1)[0])
            loss.backward()
            optimizer.step()
    

    
    

In [ ]:
#train_iters = iter(train_iter)

In [ ]:
#wee = next(train_iters)

In [ ]:
#wee.question_text.shape